In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

from v1sh_model.inputs.visualize import visualize_input, visualize_output
from v1sh_model.models.V1_model_1 import V1_model_1 as V1_model

In [ ]:
# Instantiate and test the FullModel
seed = 42
model = V1_model(seed=seed, alpha_x=1.0, alpha_y=1.0)
T = 12 
dt = 0.01

# 1. Leaking out in length or width

In [ ]:
def half_infinitly_line(N_x = 4, N_y = 2, I_input = 1.5):
    A = np.full((N_y, N_x), np.pi / 2)
    C = np.zeros((N_y, N_x))
    C[int(N_y / 2), int(N_x / 2):] = I_input
    return A, C

In [ ]:
N_y = 100
N_x = 400
A, C = half_infinitly_line(N_x=N_x, N_y=N_y, I_input=1.5)
visualize_input(A, C, dpi = 300)
plt.show()

X_gen, Y_gen, I = model.simulate(A, C, dt=dt, T=T, verbose=False, noisy=False, mode="wrap")

In [ ]:
model_output = model.g_x(X_gen.mean(axis=0)) # N_y x N_x x K
C_out = model_output # .max(axis=-1)  # N_y x N_x
argmax_angle_indices = model_output.argmax(axis=-1)  # N_y x N_x
A_out = np.broadcast_to(model.M, (N_y, N_x, model.K))
visualize_output(A_out[45:-45, 175:-175], C_out[45:-45, 175:-175] / 2, verbose=False)
plt.show()

Observation:
- no leaking out for low inputs (1.5) and averaging x(t) over time first, then taking g_x; as analyzed and in Fig. 5.61.B
-> do I see activation of neurons in same hypercolumn though?
- leaking out in length and width for higher input and averaging after taking g_x

# 2. Filling in

Next to 
a) colinear facilitation = filling-in gaps of a line through excitation of neighboring neurons
also 
b) non-conspicious gaps due to dominating iso-orientation supression in iso-oriented textures

Show b) here..

In [ ]:
def texture_with_gaps(N_x = 4, N_y = 2, I_input = 1.5, gap_prob = 0.1, seed = None):
    A = np.full((N_y, N_x), np.pi / 4)
    rng = np.random.default_rng(seed)
    C = rng.binomial(1, 1 - gap_prob, size = (N_y, N_x)) * I_input
    return A, C

A, C = texture_with_gaps(N_x=20, N_y=20, I_input=1.5, gap_prob=0.05, seed=None)
visualize_input(A, C, dpi = 400)
plt.show()

In [ ]:
X_gen_fill, _, _ = model.simulate(A, C, dt=dt, T=T, verbose=False, noisy=False, mode="wrap")

In [ ]:
model_output = model.g_x(X_gen_fill).mean(axis=0) # N_y x N_x x K
C_out = model_output # .max(axis=-1)  # N_y x N_x
argmax_angle_indices = model_output.argmax(axis=-1)  # N_y x N_x
A_out = np.broadcast_to(model.M, (20, 20, model.K))
visualize_output(A_out, C_out * 10, verbose=False)
plt.show()

Observation:
- small but existing values in gaps for mean after g_x
- but no output (for the same input value as in the previous figure) else

-> check Zhaoping 2001 to compare and get input contrast values